In [1]:
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch

In [18]:
epochs = 10
range_min = 0
range_max = 9999999999
batch_size = 4 
num_classes = 2
num_train_samples = 100
num_test_samples = 100
num_digits = 10

In [20]:
temp = np.random.rand(num_train_samples).astype(np.float64) * range_max
temp = np.floor(temp)
x_train = np.empty((num_train_samples,3)).astype(np.float64)
x_train[:,0] = temp
x_train[:num_train_samples // 2,1] = 1.0  
x_train[num_train_samples // 2:,1] = -1.0
x_train[:,2] = num_digits
np.random.shuffle(x_train)
y_train = x_train[:,0] * x_train[:,1]
print(x_train.shape)
print(y_train.shape)

(100, 3)
(100,)


In [19]:
temp = np.random.randint(low=range_min,high = range_max+1,size = (num_train_samples,))
x_train = np.empty((num_train_samples,3))
x_train[:,0] = temp
x_train[:num_train_samples // 2,1] = 1.0  
x_train[num_train_samples // 2:,1] = -1.0
x_train[:,2] = num_digits
np.random.shuffle(x_train)
y_train = x_train[:,0] * x_train[:,1]
print(x_train.shape)
print(y_train.shape)

ValueError: high is out of bounds for int32

In [51]:
train_data = torch.from_numpy(x_train).type(torch.float64)
train_labels = torch.from_numpy(y_train).type(torch.float64)
# test_data = torch.from_numpy(x_test).type(torch.FloatTensor)
# test_labels = torch.from_numpy(y_test).type(torch.LongTensor)
torch.set_printoptions(precision = 15)
torch.set_default_dtype(torch.float64)

In [69]:
# This model subtracts two numbers
class Net_sub(nn.Module):
    def __init__(self):
        super(Net_sub,self).__init__()
        self.fc1 = nn.Linear(2,1,bias=False)
        self.fc1.weight.data = torch.tensor([[1,-1]]).type(torch.float64)
    
    def forward(self,x):
        x = self.fc1(x)
        return x
    
    def predict(self,x):
        return self.forward(x).type(torch.float64)
    
# model_sub = torch.load('model_sub').double()
# model_sub = torch.load('model_sub').type(torch.float64)
model_sub = Net_sub().type(torch.float64)

In [70]:
# This model will separate a number into digits
# input: 2 inputs - (number,num_digits)
# output: separated digits (num_digits,)  

class Net_separate(nn.Module):
    def __init__(self,model_sub):
        super(Net_separate,self).__init__()
        
        self.divide_by_10 = nn.Linear(1,1,bias=False).type(torch.float64)
        self.divide_by_10.weight.data = torch.tensor([[0.1]]).type(torch.float64)

        self.multiply_by_10 = nn.Linear(1,1,bias=False)
        self.multiply_by_10.weight.data = torch.tensor([[10]]).type(torch.float64)
        
        self.model_sub = model_sub
    
    def forward(self,x):
        input_number = x[:,0].unsqueeze(-1)
        num_digits = x[:,1].unsqueeze(-1)
        concat = []
        data = input_number
        for i in range(num_digits[0].type(torch.IntTensor)):
            temp_data = self.divide_by_10(data)
#             print(temp_data)
            temp_data = temp_data.floor()
            temp = self.multiply_by_10(temp_data)
            concat_temp = torch.cat((data,temp),1)
            digit = self.model_sub.predict(concat_temp)
            
            concat.insert(0,digit)
            data = temp_data
        
        out = torch.cat(concat,1)
        return out
    
    def predict(self,x):
        return self.forward(x).type(torch.float64)
    
# model_separate = torch.load('model_separate').double()  
# model_separate = torch.load('model_separate').type(torch.float64)
model_separate = Net_separate(model_sub).type(torch.float64)

In [71]:
# this model mulltiplies a single digit number with a sign (1,-1)
# input: 2 numbers (2,)
# output: single number 
class Net_mul_sign_0_9(nn.Module):
    def __init__(self):
        super(Net_mul_sign_0_9,self).__init__()
        self.fc1 = nn.Linear(2,10)
        self.fc2 = nn.Linear(10,1)
    
    def forward(self,x):
        x = F.softsign(self.fc1(x))
        x = self.fc2(x)
        return x
    
    def predict(self,x):
        out = self.forward(x)
        return torch.round(out).type(torch.float64)

# model_mul_sign_0_9 = torch.load('model_mul_sign_0_9').double()
model_mul_sign_0_9 = torch.load('model_mul_sign_0_9').type(torch.float64)

In [72]:
# this model mulltiplies a number with a sign (1,-1)
# input: 3 numbers (3,) - [number,sign,num_digits]
# output: single number 
class Net_mul_sign_abs(nn.Module):
    def __init__(self,model_separate,model_mul_sign_0_9):
        super(Net_mul_sign_abs,self).__init__()
        self.model_separate = model_separate
        self.model_mul_sign_0_9 = model_mul_sign_0_9
        self.multiply_2 = nn.Linear(1,1,bias = False)
        self.multiply_2.weight.data = torch.tensor([[2.0]]).type(torch.float64)
        self.layers = []
    
    def forward(self,x):
        number = x[:,0].unsqueeze(-1)
        sign = x[:,1].unsqueeze(-1)
        num_digits = x[:,2].unsqueeze(-1)
        
        output_digits = self.multiply_2(num_digits)
        concat = torch.cat((number,output_digits),1)
        sep = self.model_separate.predict(concat)
#         print(sep)
        products = []
        i = 0
        j = sep.shape[-1]
        for n in range(j,0,-1):
            temp = sep[:,n-1:n]
            concat = torch.cat((temp,sign),1)
            prod = self.model_mul_sign_0_9.predict(concat)
            self.layers.append(nn.Linear(1,1,bias=False))
            self.layers[-1].weight.data = torch.tensor([[10.0**i]]).type(torch.float64)
            prod = self.layers[-1](prod)
            products.append(prod)
            i+=1
        
        concat = torch.cat(products,1)
        self.layers.append(nn.Linear(1,1,bias=False))
        self.layers[-1].weight.data = np.repeat(torch.tensor([[1.0]]),len(products),axis = -1).type(torch.float64)
        out = self.layers[-1](concat)
        return out
    
    def predict(self,x):
        out = self.forward(x)
        return out.type(torch.float64)
    
# model_mul_sign_abs = torch.load('model_mul_sign_abs').double()

In [73]:
net = Net_mul_sign_abs(model_separate,model_mul_sign_0_9).type(torch.float64)

In [74]:
print(train_data[0:1])
print(train_labels[0:1])
net.forward(train_data[0:1])

tensor([[ 7.016714045000000e+09, -1.000000000000000e+00,  1.000000000000000e+01]])
tensor([-7.016714045000000e+09])
tensor([[7.016714045000000e+08]], grad_fn=<MmBackward>)
tensor([[70167140.400000005960464]], grad_fn=<MmBackward>)
tensor([[7016714.]], grad_fn=<MmBackward>)
tensor([[701671.400000000023283]], grad_fn=<MmBackward>)
tensor([[70167.100000000005821]], grad_fn=<MmBackward>)
tensor([[7016.700000000000728]], grad_fn=<MmBackward>)
tensor([[701.600000000000023]], grad_fn=<MmBackward>)
tensor([[70.100000000000009]], grad_fn=<MmBackward>)
tensor([[7.]], grad_fn=<MmBackward>)
tensor([[0.700000000000000]], grad_fn=<MmBackward>)
tensor([[0.]], grad_fn=<MmBackward>)
tensor([[0.]], grad_fn=<MmBackward>)
tensor([[0.]], grad_fn=<MmBackward>)
tensor([[0.]], grad_fn=<MmBackward>)
tensor([[0.]], grad_fn=<MmBackward>)
tensor([[0.]], grad_fn=<MmBackward>)
tensor([[0.]], grad_fn=<MmBackward>)
tensor([[0.]], grad_fn=<MmBackward>)
tensor([[0.]], grad_fn=<MmBackward>)
tensor([[0.]], grad_fn=<MmBac

tensor([[-7.016714045000000e+09]], grad_fn=<MmBackward>)

In [75]:
torch.set_printoptions(precision=15)
out = net.predict(train_data)
temp = train_labels.numpy()
out = out.detach().numpy()
correct = 0
for i in range(out.shape[0]):
    if out[i] == temp[i]:
        correct+=1
    else:
        print('wrong example :',i)
        print(train_data[i])
        print(temp[i])
        print(out[i])
print(correct)

tensor([[7.016714045000000e+08],
        [1.848785309000000e+08],
        [7.175172950000000e+08],
        [6.063519540000000e+08],
        [5.702586941000000e+08],
        [4.544885421000000e+08],
        [6.812792462000000e+08],
        [5.288996730000000e+07],
        [2.738975166000000e+08],
        [1.030470925000000e+08],
        [2.346167237000000e+08],
        [6.386227750000000e+07],
        [7.504543222000000e+08],
        [4.644322012000000e+08],
        [3.809736000000000e+06],
        [3.120372273000000e+08],
        [4.747838579000000e+08],
        [7.037941298000001e+08],
        [4.108513707000000e+08],
        [9.629269127000000e+08],
        [3.236391900000000e+06],
        [7.920251713000001e+08],
        [8.776859565000000e+08],
        [1.797074262000000e+08],
        [6.311430970000000e+07],
        [4.506420689000000e+08],
        [6.798314456000000e+08],
        [6.037156435000000e+08],
        [9.646628444000001e+08],
        [2.574740781000000e+08],
        [2

        [ 307422.700000000011642]], grad_fn=<MmBackward>)
tensor([[701671.400000000023283],
        [184878.500000000000000],
        [717517.200000000069849],
        [606351.900000000023283],
        [570258.599999999976717],
        [454488.500000000000000],
        [681279.200000000069849],
        [ 52889.900000000001455],
        [273897.500000000000000],
        [103047.000000000000000],
        [234616.700000000011642],
        [ 63862.200000000004366],
        [750454.300000000046566],
        [464432.200000000011642],
        [  3809.700000000000273],
        [312037.200000000011642],
        [474783.800000000046566],
        [703794.100000000093132],
        [410851.300000000046566],
        [962926.900000000023283],
        [  3236.300000000000182],
        [792025.100000000093132],
        [877685.900000000023283],
        [179707.400000000023283],
        [ 63114.300000000002910],
        [450642.000000000000000],
        [679831.400000000023283],
        [603715.59999999

        [ 307.400000000000034]], grad_fn=<MmBackward>)
tensor([[701.600000000000023],
        [184.800000000000011],
        [717.500000000000000],
        [606.300000000000068],
        [570.200000000000045],
        [454.400000000000034],
        [681.200000000000045],
        [ 52.800000000000004],
        [273.800000000000011],
        [103.000000000000000],
        [234.600000000000023],
        [ 63.800000000000004],
        [750.400000000000091],
        [464.400000000000034],
        [  3.800000000000000],
        [312.000000000000000],
        [474.700000000000045],
        [703.700000000000045],
        [410.800000000000011],
        [962.900000000000091],
        [  3.200000000000000],
        [792.000000000000000],
        [877.600000000000023],
        [179.700000000000017],
        [ 63.100000000000001],
        [450.600000000000023],
        [679.800000000000068],
        [603.700000000000045],
        [964.600000000000023],
        [257.400000000000034],
        [284.60

        [0.300000000000000]], grad_fn=<MmBackward>)
tensor([[0.700000000000000],
        [0.100000000000000],
        [0.700000000000000],
        [0.600000000000000],
        [0.500000000000000],
        [0.400000000000000],
        [0.600000000000000],
        [0.000000000000000],
        [0.200000000000000],
        [0.100000000000000],
        [0.200000000000000],
        [0.000000000000000],
        [0.700000000000000],
        [0.400000000000000],
        [0.000000000000000],
        [0.300000000000000],
        [0.400000000000000],
        [0.700000000000000],
        [0.400000000000000],
        [0.900000000000000],
        [0.000000000000000],
        [0.700000000000000],
        [0.800000000000000],
        [0.100000000000000],
        [0.000000000000000],
        [0.400000000000000],
        [0.600000000000000],
        [0.600000000000000],
        [0.900000000000000],
        [0.200000000000000],
        [0.200000000000000],
        [0.100000000000000],
        [0.900000000

        [0.]], grad_fn=<MmBackward>)
tensor([[0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.

        [0.]], grad_fn=<MmBackward>)
100


In [47]:
torch.save(net,'model_mul_sign_abs')

C:\Users\Ammar\.conda\envs\neuralnets\lib\site-packages\torch\serialization.py:256: UserWarning: Couldn't retrieve source code for container of type Net_mul_sign_abs. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
C:\Users\Ammar\.conda\envs\neuralnets\lib\site-packages\torch\serialization.py:256: UserWarning: Couldn't retrieve source code for container of type Net_separate. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
C:\Users\Ammar\.conda\envs\neuralnets\lib\site-packages\torch\serialization.py:256: UserWarning: Couldn't retrieve source code for container of type Net_sub. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
C:\Users\Ammar\.conda\envs\neuralnets\lib\site-packages\torch\serialization.py:256: UserWarning: Couldn't retrieve source code for container of type Net_mul_sign_0_9. It won't be checked for correctness upo

In [67]:
class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        self.fc1 = nn.Linear(1,1,bias=False)
        self.fc1.weight.data = torch.tensor([[0.1]]).type(torch.float64)
    
    def forward(self,x):
        x = self.fc1(x)
        return x
    
model = Net().type(torch.float64)

In [68]:
input_data = np.array([[99397544]])
input_data = torch.from_numpy(input_data).type(torch.float64)
print(model.forward(input_data))

tensor([[9939754.400000000372529]], grad_fn=<MmBackward>)
